In [61]:
from dotenv import load_dotenv

import os

load_dotenv()
ASU_key = os.environ.get("ASU_key") 

import requests
endpoint_url = 'https://api-dev-poc.aiml.asu.edu/eval'
headers = {
    "Authorization": f"Bearer {ASU_key}",
    "Content-Type": "application/json"
}


In [62]:
import pandas as pd
df = pd.read_csv('qa_results_evaluated_v2.csv')
df.head()
df['ragas_bleu'] = 0.0
df['ragas_precision_reference'] = 0.0
df['ragas_faith'] = 0.0
# df['ragas_fact'] = 0.0
df['regas_sem_sim'] = 0.0
df['regas_rogue'] = 0.0

In [63]:
questiony = df['question'].iloc[0]
answery = df['answer'].iloc[0]
contexty = df['page_text'].iloc[0]

##################################
### Bluescore
payload_blue = {
    "model_provider": "openai",
    "model_name": "gpt-4o",
    "metric": "bleuscore",
    "parameters": {
        "user_input": questiony,
        "response": answery,
        "reference": contexty,
    }
}
try:
    response = requests.post(endpoint_url, headers=headers, json=payload_blue)
    response.raise_for_status()
    result = response.json()
    print("result:", result)
except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

#### SAVE BLEU score
df.loc[0,'ragas_bleu'] = result['score']

##################################
### Context Precision With Reference
payload_precision_reference = {
    "model_provider": "openai",
    "model_name": "gpt-4o",
    "metric": "context_precision_with_reference",
    "parameters": {
        "user_input": questiony,
        "reference": answery,
        "retrieved_contexts": [
            contexty,
        ]
    }
}
try:
    response = requests.post(endpoint_url, headers=headers, json=payload_precision_reference)
    response.raise_for_status()
    result = response.json()
    print("result:", result)
except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

#### SAVE Precision With Reference score
df.loc[0,'ragas_precision_reference'] = result['score']

##################################
### RAGAS Faithfulness
payload_ragas_faith = {
    "model_provider": "openai",
    "model_name": "gpt-4o",
    "metric": "faithfulness",
    "parameters": {
        "user_input": questiony,
        "response": answery,
        "retrieved_contexts": [
            contexty,
        ]
    }
}
try:
    response = requests.post(endpoint_url, headers=headers, json=payload_ragas_faith)
    response.raise_for_status()
    result = response.json()
    print("result:", result)
except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

#### SAVE RAGAS Faithfulness score
df.loc[0,'ragas_faith'] = result['score']

##################################
# ### Factual correctness
# print("Factual Correctness")
# payload_fact = {
#     "model_provider": "openai",
#     "model_name": "gpt-4o",
#     "metric": "factual_correctness",
#     "parameters": {
#         "response": str(answery),
#         "reference": "The Eiffel Tower is located in Paris. I has a height of 1000ft."
#     }
# }
# try:
#     response = requests.post(endpoint_url, headers=headers, json=payload_fact)
#     response.raise_for_status()
#     result = response.json()
#     print("result:", result)
# except requests.exceptions.RequestException as e:
#     print(f"API request error: {e}")
# except Exception as e:
#     print(f"Unexpected error: {e}")

# #### SAVE Factual correctness score
# df.loc[0,'ragas_fact'] = result['score']

##################################
### Semantic similarity
payload_sem_sim = {
    "model_provider": "openai",
    "model_name": "gpt-4o",
    "metric": "semantic_similarity",
    "parameters": {
        "response": answery,
        "reference": contexty
    }
}
try:
    response = requests.post(endpoint_url, headers=headers, json=payload_sem_sim)
    response.raise_for_status()
    result = response.json()
    print("result:", result)
except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

#### SAVE Factual correctness score
df.loc[0,'regas_sem_sim'] = result['score']

##################################
### Rouge score
payload_rogue = {
    "model_provider": "openai",
    "model_name": "gpt-4o",
    "metric": "rouge_score",
    "parameters": {
        "response": answery,
        "reference": contexty
    }
}
try:
    response = requests.post(endpoint_url, headers=headers, json=payload_rogue)
    response.raise_for_status()
    result = response.json()
    print("result:", result)
except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

#### SAVE Factual correctness score
df.loc[0,'regas_rogue'] = result['score']

result: {'metric': 'bleuscore', 'score': 0.14694106251955755}
result: {'metric': 'context_precision_with_reference', 'score': 0.9999999999}
result: {'metric': 'faithfulness', 'score': 1.0}
result: {'metric': 'semantic_similarity', 'score': 0.9538778530987497}
result: {'metric': 'rouge_score', 'score': 0.1603563474387528}


In [64]:
{
    "model_provider": "openai",
    "model_name": "gpt-4o",
    "metric": "factual_correctness",
    "parameters": {
        "response": "The Eiffel Tower is located in Paris.",
        "reference": "The Eiffel Tower is located in Paris. I has a height of 1000ft."
    }
}

{'model_provider': 'openai',
 'model_name': 'gpt-4o',
 'metric': 'factual_correctness',
 'parameters': {'response': 'The Eiffel Tower is located in Paris.',
  'reference': 'The Eiffel Tower is located in Paris. I has a height of 1000ft.'}}

In [69]:
payload_fact

{'model_provider': 'openai',
 'model_name': 'gpt-4o',
 'metric': 'factual_correctness',
 'parameters': {'response': 'The Bachelor of Science  BS  program in accountancy at the W  P  Carey School of Accountancy is designed to provide students with a comprehensive skill set that is essential for success in the accounting profession  This program emphasizes a blend of technical knowledge and practical skills  ensuring that graduates are well prepared to meet the demands of the real world business environment   One of the primary skills that the program aims to equip students with is critical thinking  This skill ',
  'reference': 'Program Description  The W  P  Carey School of Accountancy works closely with its Professional Advisory Board to create a learning environment that emulates the real world by exploring applied processes and strategies  The BS program in accountancy prepares students by equipping them with critical thinking  team building  communication and information technology

In [72]:
import re

# answery1 = re.sub(r'[^a-zA-Z0-9]', ' ', answery)
# contexty1 = re.sub(r'[^a-zA-Z0-9]', ' ', contexty)

payload_fact = {
    "model_provider": "openai",
    "model_name": "gpt-4o",
    "metric": "factual_correctness",
    "parameters": {
        "response": str(answery),
        "reference": str(contexty)
    }
}


try:
    response = requests.post(endpoint_url, headers=headers, json=payload_fact)
    response.raise_for_status()
    result = response.json()
    print("result:", result)
except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

result: {'metric': 'factual_correctness', 'score': 0.53}


In [67]:
df.head()

,url,depth,title,topic,word_count,char_count,page_text,question,answer,TFIDF_Cosine_Similarity,Fuzzy_Similarity,BLEU_Score,SBERT_Similarity,Combined_Similarity_Score,ragas_bleu,ragas_precision_reference,ragas_faith,regas_sem_sim,regas_rogue
0,https://degrees.apps.asu.edu/bachelors/major/A...,1,"Accountancy, BS",Academic Programs & Courses,57,418,Program Description: The W. P. Carey School of...,What skills does the BS program in accountanc...,The Bachelor of Science (BS) program in accoun...,0.546,0.28,0.032,0.771,0.407,0.146941,1.0,1.0,0.953878,0.160356
1,https://degrees.apps.asu.edu/bachelors/major/A...,1,"Accountancy, BS",Academic Programs & Courses,57,418,Program Description: The W. P. Carey School of...,Who collaborates with the W. P. Carey School ...,The W. P. Carey School of Accountancy at Arizo...,0.649,0.29,0.097,0.821,0.464,0.000000,0.0,0.0,0.000000,0.000000
2,https://degrees.apps.asu.edu/bachelors/major/A...,1,"Accountancy, BS",Academic Programs & Courses,57,418,Program Description: The W. P. Carey School of...,What type of learning environment does the W....,The W. P. Carey School of Accountancy at Arizo...,0.560,0.21,0.038,0.821,0.464,0.000000,0.0,0.0,0.000000,0.000000
3,https://degrees.apps.asu.edu/bachelors/major/A...,1,"Accountancy, BS",Academic Programs & Courses,57,418,Program Description: The W. P. Carey School of...,What are some of the key areas of focus in th...,The Bachelor of Science (BS) program in Accoun...,0.574,0.14,0.042,0.736,0.373,0.000000,0.0,0.0,0.000000,0.000000
4,https://degrees.apps.asu.edu/bachelors/major/A...,1,"Actuarial Science, BS",Academic Programs & Courses,97,692,Program Description: Actuarial science student...,What tools do actuarial science students lear...,Actuarial science students at Arizona State Un...,0.712,0.10,0.113,0.685,0.402,0.000000,0.0,0.0,0.000000,0.000000
